In [1]:
import numpy as np
import torch
import torch.nn as nn
from torch.autograd import Variable
import os
import cv2
import glob
from Models.LargePNet import *
import tifffile
from Models.Discriminator import *
from Utils.TrainerRaGAN32 import *
from Utils.AppendLoad import *

C:\Users\Dell\.conda\envs\DL\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load Raw .npz data
Data_dir = r'F:\Datasets\STED_deblurring\MitoInner\Training'
loaded_data = np.load(Data_dir + '\MitoDeblurdata.npz')
X_train = loaded_data['X_train']  
y_train = loaded_data['y_train']
X_val = loaded_data['X_val']
y_val = loaded_data['y_val']
log_dir = Data_dir + r'\logfile'
if not os.path.exists(log_dir):
    os.mkdir(log_dir)
# Create training and validation data
X_train = torch.tensor(X_train, dtype = torch.float32).unsqueeze(1)
y_train = torch.tensor(y_train, dtype = torch.float32).unsqueeze(1)
X_val = torch.tensor(X_val, dtype = torch.float32).unsqueeze(1)
y_val = torch.tensor(y_val, dtype = torch.float32).unsqueeze(1)
print(X_train.shape)
print(y_train.shape)

''' Another loading method: load .tif image file
head_dir = r"E:\SMLM\MT\512data\Training"
Training_GT_path = head_dir + '\\' + r'GT\*.tif'
Training_Raw_path = head_dir + '\\' + r'Noisy\*.tif'
Val_GT_path = head_dir + '\\' + r'ValGT\*.tif'
Val_Raw_path = head_dir + '\\' + r'ValNoisy\*.tif'

X_train = AppendLoad(Training_Raw_path)
y_train = AppendLoad(Training_GT_path)
X_val = AppendLoad(Val_Raw_path)
y_val = AppendLoad(Val_GT_path)

print("X_train.shape",X_train.shape)
print("y_train.shape",y_train.shape)
print("X_val.shape",X_val.shape)
print("y_val.shape",y_val.shape)

train_data = Data.TensorDataset(X_train,y_train)
val_data = Data.TensorDataset(X_val,y_val)
'''

torch.Size([288, 1, 1024, 1024])
torch.Size([288, 1, 1024, 1024])


' Another loading method: load .tif image file\nhead_dir = r"E:\\SMLM\\MTŊdata\\Training"\nTraining_GT_path = head_dir + \'\\\' + r\'GT\\*.tif\'\nTraining_Raw_path = head_dir + \'\\\' + r\'Noisy\\*.tif\'\nVal_GT_path = head_dir + \'\\\' + r\'ValGT\\*.tif\'\nVal_Raw_path = head_dir + \'\\\' + r\'ValNoisy\\*.tif\'\n\nX_train = AppendLoad(Training_Raw_path)\ny_train = AppendLoad(Training_GT_path)\nX_val = AppendLoad(Val_Raw_path)\ny_val = AppendLoad(Val_GT_path)\n\nprint("X_train.shape",X_train.shape)\nprint("y_train.shape",y_train.shape)\nprint("X_val.shape",X_val.shape)\nprint("y_val.shape",y_val.shape)\n\ntrain_data = Data.TensorDataset(X_train,y_train)\nval_data = Data.TensorDataset(X_val,y_val)\n'

In [3]:
# Augment data
img_num = X_train.shape[0]
img_num_val = X_val.shape[0]
wanted_num = 2000
aimingsize = 512
background_control_max = 0.05
background_control_mean = 0.02 # typically 0.015-0.03

X_train_expand = []
y_train_expand = []
X_val_expand = []
y_val_expand = []
crop_num = wanted_num // img_num

count = 0
for i in tqdm(range(img_num), desc="Processing images"):  
    raw = X_train[i,0,...]
    gt = y_train[i,0,...]
    raw = np.expand_dims(raw,axis=2)
    gt = np.expand_dims(gt,axis=2)
    for j in range (crop_num):
        Augraw, Auggt = DataAug(raw,gt,aimingsize)
        if Auggt.max()>background_control_max:
            if Auggt.mean()>background_control_mean:
                X_train_expand.append(Augraw.squeeze())
                y_train_expand.append(Auggt.squeeze())
                count = count + 1
X_train = np.array(X_train_expand)
y_train = np.array(y_train_expand)

count = 0
for i in tqdm(range(img_num_val), desc="Processing images"):  
    raw = X_val[i,0,...]
    gt = y_val[i,0,...]
    raw = np.expand_dims(raw,axis=2)
    gt = np.expand_dims(gt,axis=2)
    for j in range (crop_num//2):
        Augraw, Auggt = DataAug(raw,gt,aimingsize)
        if Auggt.max()>background_control_max:
            if Auggt.mean()>background_control_mean:
                X_val_expand.append(Augraw.squeeze())
                y_val_expand.append(Auggt.squeeze())
                count = count + 1
X_val = np.array(X_val_expand)
y_val = np.array(y_val_expand)

X_train = torch.tensor(X_train, dtype = torch.float32).unsqueeze(1)
y_train = torch.tensor(y_train, dtype = torch.float32).unsqueeze(1)
X_val = torch.tensor(X_val, dtype = torch.float32).unsqueeze(1)
y_val = torch.tensor(y_val, dtype = torch.float32).unsqueeze(1)

print(X_train.shape)
print(y_train.shape)

train_data = Data.TensorDataset(X_train,y_train)
val_data = Data.TensorDataset(X_val,y_val)

Processing images: 100%|██████████| 48/48 [00:01<00:00, 42.07it/s]


torch.Size([1580, 1, 512, 512])
torch.Size([1580, 1, 512, 512])


In [5]:
class Options:  
    def __init__(self):  
        self.LR = 0.001
        self.batchsize = 1 
        self.epoch_num = 50 
        self.MSE_weight = 0
        self.MAE_weight = 1   
        self.SSIM_weight = 0 
        self.epoch_critic = 10
        self.useinit = 0
        self.ModelType = 1   
        self.loss_content = 0.006
        self.loss_GAN = 0.001
        self.D_decay = 500   # Decay of dicriminator leraning rate over generator
        self.warmupepoch = 0
        self.Dinterval = 1
        self.StepLR = 1
        self.ExpoLR = 0
        self.val = 1
        self.test = 0
        self.use_dir = 0
        self.use_norm = 1
        self.val_data_dir = ''
        self.innerpoint = []
        self.instanceimage = ''
        self.val_start = 10
opt = Options()

In [6]:
netG = LargePNet(1,1,1,25,4)
hr_shape = (512,512)
netD = MultiScaleDiscriminator(input_shape = (1, *hr_shape))
save_path = Data_dir+r'\logfile\LargePGAN'
TrainerRaGAN32(netG, netD, train_data, val_data, save_path, opt)

drop path: Identity()
drop path: Identity()
drop path: Identity()


C:\Users\Dell\.conda\envs\DL\lib\site-packages\torch\functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
C:\Users\Dell\.conda\envs\DL\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\Dell\.conda\envs\DL\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


0


Training epoch: 100%|██████████| 1580/1580 [09:17<00:00,  2.83it/s, Epoch: 1, Loss: 0.020300325006246567]


Epoch: 0, Loss_D: 0.3889, Loss_G: 0.0438, D(x): 0.3889, D(z): 0.3889
Epoch: 0, Val Loss: 0.001099, Val MSE: 0.001099, Val MAE: 0.018017, Val SSIM: 0.923970
1


...

Epoch: 9, Loss_D: 0.4074, Loss_G: 0.0200, D(x): 0.4074, D(z): 0.4074
Epoch: 9, Val Loss: 0.001230, Val MSE: 0.001230, Val MAE: 0.018632, Val SSIM: 0.927455
10


Training epoch: 100%|██████████| 1580/1580 [08:52<00:00,  2.97it/s, Epoch: 11, Loss: 0.033062901347875595]


Epoch: 10, Loss_D: 0.4196, Loss_G: 0.0197, D(x): 0.4196, D(z): 0.4196
Epoch: 10, Val Loss: 0.000934, Val MSE: 0.000934, Val MAE: 0.017245, Val SSIM: 0.926208
11


Training epoch:  12%|█▏        | 195/1580 [01:10<08:17,  2.78it/s, Epoch: 12, Loss: 0.02089753746986389] 


KeyboardInterrupt: 